In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/burns-trauma.csv")
cardio=pd.read_csv("data2/cardiovascular.csv")
endocrine=pd.read_csv("data2/endocrine.csv")
gastrointestinal=pd.read_csv("data2/gastrointestinal.csv")
hematology=pd.read_csv("data2/hematology.csv")
infectiousdiseases=pd.read_csv("data2/infectious diseases.csv")
neurologic=pd.read_csv("data2/neurologic.csv")
oncology=pd.read_csv("data2/oncology.csv")
pulmonary=pd.read_csv("data2/pulmonary.csv")
renal=pd.read_csv("data2/renal.csv")
surgery=pd.read_csv("data2/surgery.csv")
toxicology=pd.read_csv("data2/toxicology.csv")
burns = pd.concat([burns, cardio, endocrine, gastrointestinal, hematology, infectiousdiseases,
          neurologic, oncology, pulmonary, renal, surgery, toxicology]).copy()

In [3]:
cvp=pd.read_csv("CVP.csv")

In [4]:
burns_merged = pd.merge(cvp[['patientunitstayid','val']], burns, how='right', on='patientunitstayid')

In [5]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [6]:
burns_merged_2 = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns_merged, how='right', on='patientunitstayid')

In [7]:
burns_merged_2['mods_ratio'] = 0

In [8]:
burns_merged_2

,patientunitstayid,pao2,fio2,val,gender,age,ethnicity,apacheadmissiondx,admissionheight,hospitaladmitoffset,...,WBC x 1000,platelets x 1000,RBC,bicarbonate,MCV,MCHC,MCH,RDW,anion gap,mods_ratio
0,143103,346.0,100.0,5.333333,Female,46.0,Asian,Head only trauma,167.6,-3.0,...,19.500000,258.000000,3.980000,20.0,85.900000,35.400000,30.4,13.5,NaN,0
1,143258,-1.0,-1.0,NaN,Male,90.0,Caucasian,"Rhythm disturbance (atrial, supraventricular)",180.3,0.0,...,8.600000,175.000000,3.480000,28.0,102.000000,33.500000,34.2,12.1,NaN,0
2,143274,-1.0,-1.0,NaN,Male,22.0,Caucasian,Head/extremity trauma,195.6,-179.0,...,14.500000,218.000000,4.440000,30.0,84.000000,32.400000,27.3,12.7,NaN,0
3,145180,-1.0,-1.0,NaN,Female,77.0,-,"Extremity only trauma, surgery for",167.6,-1891.0,...,5.400000,173.000000,3.290000,25.0,94.200000,32.300000,30.4,13.7,NaN,0
4,146784,80.0,40.0,NaN,Male,50.0,Caucasian,Spinal/face trauma,177.8,-2.0,...,14.000000,164.000000,3.660000,25.0,93.400000,33.600000,31.4,12.9,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196174,3351551,-1.0,-1.0,NaN,Male,42.0,Caucasian,"Overdose, sedatives, hypnotics, antipsychotics...",188.0,-80.0,...,17.900000,326.000000,4.270000,26.0,96.000000,35.000000,NaN,NaN,NaN,0
196175,3352119,304.0,100.0,NaN,Male,57.0,Caucasian,"Overdose, sedatives, hypnotics, antipsychotics...",188.0,0.0,...,13.250000,241.000000,5.165000,28.0,84.000000,33.000000,NaN,NaN,NaN,0
196176,3352193,98.0,30.0,NaN,Female,19.0,Other/Unknown,Head/face trauma,157.5,-188.0,...,9.882586,205.286588,4.030237,23.0,91.320663,33.395358,NaN,NaN,NaN,0
196177,3352563,-1.0,-1.0,NaN,Female,34.0,Caucasian,"Overdose, sedatives, hypnotics, antipsychotics...",172.7,-231.0,...,9.200000,137.000000,3.840000,17.0,84.000000,31.000000,NaN,NaN,NaN,0


In [9]:
for i in range(len(burns_merged_2)):
    if (burns_merged_2.iloc[i]['pao2'] != -1) & (burns_merged_2.iloc[i]['fio2'] != -1):
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 300) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 226):
            burns_merged_2['mods_ratio'].iloc[i] = 1
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 225) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 151):
            burns_merged_2['mods_ratio'].iloc[i] = 2
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 150) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 76):
            burns_merged_2['mods_ratio'].iloc[i] = 3
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 0) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 75):
            burns_merged_2['mods_ratio'].iloc[i] = 4

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
def platelets(item):
    temp = 0
    if (item >= 81) & (item <= 120):
        temp += 1
    elif (51 <= item) & (item <= 80):
        temp += 2
    elif (21 <= item) & (item <= 50):
        temp += 3
    elif (20 >= item):
        temp += 4
    return temp

In [11]:
burns_merged_2['mods_platelets'] = burns_merged_2['platelets x 1000'].apply(platelets)

In [12]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [13]:
burns_merged_3 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged_2, how='right', on='patientunitstayid')

In [14]:
def bilirubin(item):
    temp = 0
    if (item >= 1.3) & (item <= 3.5):
        temp += 1
    elif (3.6 <= item) & (item <= 7.0):
        temp += 2
    elif (7.0 <= item) & (item <= 14.0):
        temp += 3
    elif (14.0 <= item):
        temp += 4
    return temp

In [15]:
burns_merged_3['mods_bilirubin'] = burns_merged_3['bilirubin'].apply(bilirubin)

In [16]:
burns_merged_3['val_fill'] = burns_merged_3['val'].fillna(burns_merged_3['val'].mean())

In [17]:
burns_merged_3['PAHR'] = (burns_merged_3['heartrate'] * burns_merged_3['val'])/burns_merged_3['meanbp']

In [18]:
burns_merged_3['PAHRF'] = (burns_merged_3['heartrate'] * burns_merged_3['val_fill'])/burns_merged_3['meanbp']

In [19]:
burns_merged_3['PAHRF'].mean()

22.62286142267902

In [20]:
def ad_pressure(item):
    temp = 0
    if (item >= 10.1) & (item <= 15):
        temp += 1
    elif (item >= 15.1) & (item <= 20):
        temp += 2
    elif (item >= 20.1) & (item <= 30):
        temp += 3
    elif (item >= 30.1):
        temp += 4
    return temp

In [21]:
burns_merged_3['mods_pressure'] = burns_merged_3['PAHR'].apply(ad_pressure)

In [22]:
burns_merged_3['GCS'] = burns_merged_3['eyes'] + burns_merged_3['motor'] + burns_merged_3['verbal']

In [23]:
def GCS(item):
    temp = 0
    if (item >= 13) & (item <= 14):
        temp += 1
    elif (10 <= item) & (item <= 12):
        temp += 2
    elif (6 <= item) & (item <= 9):
        temp += 3
    elif (6 > item):
        temp += 4
    return temp

In [24]:
burns_merged_3['mods_GCS'] = burns_merged_3['GCS'].apply(GCS)

In [25]:
def creatinine(item):
    temp = 0
    if (item >= 1.2) & (item <= 2.2):
        temp += 1
    elif (2.3 <= item) & (item <= 3.9):
        temp += 2
    elif (4 <= item) & (item <= 5.7):
        temp += 3
    elif (5.7 < item):
        temp += 4
    return temp

In [26]:
burns_merged_3['mods_creatinine'] = burns_merged_3['creatinine'].apply(creatinine)

In [27]:
burns_merged_3.columns

Index(['patientunitstayid', 'bilirubin', 'pao2', 'fio2', 'val', 'gender',
       'age', 'ethnicity', 'apacheadmissiondx', 'admissionheight',
       'hospitaladmitoffset', 'unittype', 'unitadmitsource', 'unitstaytype',
       'admissionweight', 'intubated', 'dialysis', 'temperature',
       'respiratoryrate', 'heartrate', 'meanbp', 'hematocrit',
       'actualicumortality', 'preopmi', 'preopcardiaccath', 'ptcawithin24h',
       'meds', 'verbal', 'motor', 'eyes', 'thrombolytics', 'aids',
       'hepaticfailure', 'lymphoma', 'immunosuppression', 'cirrhosis',
       'activetx', 'ima', 'midur', 'ventday1', 'oobventday1', 'oobintubday1',
       'diabetes', 'diagnosisstring', 'potassium', 'creatinine', 'sodium',
       'BUN', 'glucose', 'chloride', 'calcium', 'Hgb', 'WBC x 1000',
       'platelets x 1000', 'RBC', 'bicarbonate', 'MCV', 'MCHC', 'MCH', 'RDW',
       'anion gap', 'mods_ratio', 'mods_platelets', 'mods_bilirubin',
       'val_fill', 'PAHR', 'PAHRF', 'mods_pressure', 'GCS', 'mods_GC

In [28]:
burns_merged_3['mods_score'] = burns_merged_3[['mods_ratio', 'mods_platelets', 'mods_bilirubin',
        'mods_pressure', 'mods_GCS', 'mods_creatinine']].sum(axis=1)

In [29]:
burns_merged_3['actualicumortality_label'] = burns_merged_3['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [30]:
def mods_predict(item):
    temp = 0
    if (item >= 1) & (item <= 4):
        temp = 1.5
    elif (item >= 5) & (item <= 8):
        temp = 4.0
    elif (item >= 9) & (item <= 12):
        temp = 25
    elif (item >= 13) & (item <= 16):
        temp = 50
    elif (item >= 17) & (item <= 20):
        temp = 75
    elif (item >= 21) & (item <= 24):
        temp = 100
    return temp

In [31]:
burns_merged_3['mods_score'].max()

20

In [32]:
burns_merged_3['mods_prob'] = burns_merged_3['mods_score'].apply(mods_predict)

In [33]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_3['actualicumortality_label'].values, burns_merged_3['mods_prob'].values, pos_label=1)

In [34]:
metrics.auc(fpr, tpr)

0.7662810680068873

In [35]:
burns_merged_3['mods_prob']

0         1.5
1         1.5
2         1.5
3         1.5
4         1.5
         ... 
196174    0.0
196175    1.5
196176    1.5
196177    1.5
196178    1.5
Name: mods_prob, Length: 196179, dtype: float64

In [ ]:
18

In [ ]:
9